<a href="https://colab.research.google.com/github/edwin-19/bert_vits2_infer_py_style/blob/main/BertVits2_Infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some small infer code for bert vits2

> The main intention was get the infer code running, in a py format instead of that webui



# Well first thing first get everything ready
- Install dependency and download models
- Also clone the repo

In [1]:
!git clone https://github.com/fishaudio/Bert-VITS2.git

Cloning into 'Bert-VITS2'...
remote: Enumerating objects: 3142, done.
remote: Counting objects: 100% (1893/1893), done.
remote: Compressing objects: 100% (467/467), done.
remote: Total 3142 (delta 1558), reused 1625 (delta 1412), pack-reused 1249
Receiving objects: 100% (3142/3142), 10.33 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (1924/1924), done.


In [2]:
%cd Bert-VITS2/

/content/Bert-VITS2


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.5/751.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [8]:
!git clone https://huggingface.co/OedoSoldier/Bert-VITS2-2.3 weights/

Cloning into 'weights'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (15/15), 2.78 KiB | 710.00 KiB/s, done.
Filtering content: 100% (4/4), 1.23 GiB | 48.14 MiB/s, done.


In [9]:
import torch
import commons
from text import cleaned_text_to_sequence, get_bert

from typing import Union
from text.cleaner import clean_text
import utils

from models import SynthesizerTrn
from text.symbols import symbols

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [12]:
from infer import get_net_g, get_text
from utils import get_hparams_from_file
from IPython.display import Audio

In [15]:
hps = get_hparams_from_file("configs/config.json")
model = get_net_g("weights/G_0.pth", version='2.3', device='cpu', hps=hps)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
ERROR:utils:emb_g.weight is not in the checkpoint


In [40]:
style_weight = 0.8
sdp_ratio = 0.5
noise_scale = 0.667
noise_scale_w = 0.8
length_scale = 1

# text = "Hi this is olivia can you send me your picture, i would love to know more about you. So lets rock it baby!!"
# text = "This tech is dangerous i can easily catfish someone you know?"
text = "Can you send me your bank account number and what do you think of my voice? Does it sound cute?"
language = "EN"
sid = "空_EN"
bert, ja_bert, en_bert, phones, tones, lang_ids = get_text(
    text,
    language,
    hps,
    'cpu',
    style_text=None,
    style_weight=style_weight,
)

device = torch.device('cpu')
with torch.no_grad():
    x_tst = phones.to(device).unsqueeze(0)
    tones = tones.to(device).unsqueeze(0)
    lang_ids = lang_ids.to(device).unsqueeze(0)
    bert = bert.to(device).unsqueeze(0)
    ja_bert = ja_bert.to(device).unsqueeze(0)
    en_bert = en_bert.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([phones.size(0)]).to(device)
    del phones
    speakers = torch.LongTensor([hps.data.spk2id[sid]]).to(device)

    audio = (
        model.infer(
            x_tst,
            x_tst_lengths,
            speakers,
            tones,
            lang_ids,
            bert,
            ja_bert,
            en_bert,
            sdp_ratio=sdp_ratio,
            noise_scale=noise_scale,
            noise_scale_w=noise_scale_w,
            length_scale=length_scale,
        )[0][0, 0]
        .data.cpu()
        .float()
        .numpy()
    )

In [32]:
Audio(audio, rate=hps.data.sampling_rate)

In [35]:
Audio(audio, rate=hps.data.sampling_rate)

In [39]:
Audio(audio, rate=hps.data.sampling_rate)

In [41]:
Audio(audio, rate=hps.data.sampling_rate)

In [43]:
# Heres a list of speaker IDs change it in on top to change the voice you want
for spk, id in hps.data.spk2id.items():
  print(spk, id)

派蒙_ZH 0
纳西妲_ZH 1
凯亚_ZH 2
阿贝多_ZH 3
温迪_ZH 4
枫原万叶_ZH 5
钟离_ZH 6
荒泷一斗_ZH 7
八重神子_ZH 8
艾尔海森_ZH 9
提纳里_ZH 10
迪希雅_ZH 11
卡维_ZH 12
宵宫_ZH 13
那维莱特_ZH 14
莱依拉_ZH 15
赛诺_ZH 16
莫娜_ZH 17
诺艾尔_ZH 18
托马_ZH 19
凝光_ZH 20
林尼_ZH 21
北斗_ZH 22
柯莱_ZH 23
神里绫华_ZH 24
可莉_ZH 25
芭芭拉_ZH 26
雷电将军_ZH 27
娜维娅_ZH 28
芙宁娜_ZH 29
珊瑚宫心海_ZH 30
鹿野院平藏_ZH 31
迪奥娜_ZH 32
琴_ZH 33
五郎_ZH 34
班尼特_ZH 35
达达利亚_ZH 36
安柏_ZH 37
莱欧斯利_ZH 38
夜兰_ZH 39
妮露_ZH 40
辛焱_ZH 41
丽莎_ZH 42
珐露珊_ZH 43
魈_ZH 44
香菱_ZH 45
迪卢克_ZH 46
砂糖_ZH 47
烟绯_ZH 48
早柚_ZH 49
云堇_ZH 50
刻晴_ZH 51
重云_ZH 52
优菈_ZH 53
胡桃_ZH 54
流浪者_ZH 55
久岐忍_ZH 56
神里绫人_ZH 57
甘雨_ZH 58
戴因斯雷布_ZH 59
菲谢尔_ZH 60
白术_ZH 61
行秋_ZH 62
九条裟罗_ZH 63
夏洛蒂_ZH 64
雷泽_ZH 65
申鹤_ZH 66
荧_ZH 67
空_ZH 68
迪娜泽黛_ZH 69
凯瑟琳_ZH 70
多莉_ZH 71
坎蒂丝_ZH 72
琳妮特_ZH 73
萍姥姥_ZH 74
罗莎莉亚_ZH 75
埃德_ZH 76
爱贝尔_ZH 77
伊迪娅_ZH 78
留云借风真君_ZH 79
绮良良_ZH 80
陌生人_ZH 81
七七_ZH 82
式大将_ZH 83
瑶瑶_ZH 84
奥兹_ZH 85
菲米尼_ZH 86
米卡_ZH 87
哲平_ZH 88
浮游水蕈兽·元素生命_ZH 89
大肉丸_ZH 90
托克_ZH 91
蒂玛乌斯_ZH 92
昆钧_ZH 93
欧菲妮_ZH 94
塞琉斯_ZH 95
仆人_ZH 96
迈勒斯_ZH 97
希格雯_ZH 98
阿守_ZH 99
拉赫曼_ZH 100
杜拉夫_ZH 101
伊利亚斯_ZH 102